In [1]:
import nltk
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
latlib_df = pd.read_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_data.pkl")

In [3]:
ne_df = pd.read_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_ne_data.pkl")

In [4]:
#here's some of the data:
latlib_df.head()

,title,text,text_raw,tokens,token_count
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918


In [5]:
ne_df.head()

,name,era
0,AMMIANVS MARCELLINVS,(4th century A.D.)
1,VALERIVS MAXIMVS,(1st century A.D.)
2,GAIVS VALERIVS FLACCVS,(died c. 90 A.D.)
4,P. VERGILIVS MARO,(70 – 19 B.C.)
6,ALBIVS TIBVLLVS,(c. 54 – 19 B.C.)


Here's some basic info on the data contained in latlib_df:

In [6]:
#overall shape of the dataframe:
latlib_df.shape

(672, 5)

In [7]:
latlib_df.describe()

,token_count
count,672.000000
mean,6331.989583
std,5564.159005
min,0.000000
25%,1681.500000
50%,5578.500000
75%,8590.500000
max,38414.000000


In [8]:
latlib_df.info()
#good! no null objects

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        672 non-null    object
 1   text         672 non-null    object
 2   text_raw     672 non-null    object
 3   tokens       672 non-null    object
 4   token_count  672 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 26.4+ KB


In [9]:
#gladius:
gladius = latlib_df[(latlib_df.text.str.contains(pat = 'gladius', case=False)) | (latlib_df.text.str.contains(pat = 'gladii', case=False)) | (latlib_df.text.str.contains(pat = 'gladi', case=False)) | (latlib_df.text.str.contains(pat = 'gladio', case=False)) | (latlib_df.text.str.contains(pat = 'gladium', case=False)) | (latlib_df.text.str.contains(pat = 'gladie', case=False)) | (latlib_df.text.str.contains(pat = 'gladiorum', case=False)) | (latlib_df.text.str.contains(pat = 'gladiis', case=False)) | (latlib_df.text.str.contains(pat = 'gladios', case=False))]

In [10]:
#ferrum:
ferrum = latlib_df[(latlib_df.text.str.contains(pat = 'ferrum', case=False)) | (latlib_df.text.str.contains(pat = 'ferri', case=False)) | (latlib_df.text.str.contains(pat = 'ferro', case=False)) | (latlib_df.text.str.contains(pat = 'ferra', case=False)) | (latlib_df.text.str.contains(pat = 'ferrorum', case=False)) | (latlib_df.text.str.contains(pat = 'ferris', case=False))]

In [11]:
#mucro:
mucro = latlib_df[(latlib_df.text.str.contains(pat = 'mucro', case=False)) | (latlib_df.text.str.contains(pat = 'mucronis', case=False)) | (latlib_df.text.str.contains(pat = 'mucroni', case=False)) | (latlib_df.text.str.contains(pat = 'mucronem', case=False)) | (latlib_df.text.str.contains(pat = 'mucrone', case=False)) | (latlib_df.text.str.contains(pat = 'mucrones', case=False)) | (latlib_df.text.str.contains(pat = 'mucronum', case=False)) | (latlib_df.text.str.contains(pat = 'mucronibus', case=False))]

In [12]:
#ensis:
ensis = latlib_df[(latlib_df.text.str.contains(pat = 'ensis', case=False)) | (latlib_df.text.str.contains(pat = 'ensi', case=False)) | (latlib_df.text.str.contains(pat = 'ensem', case=False)) | (latlib_df.text.str.contains(pat = 'ense', case=False)) | (latlib_df.text.str.contains(pat = 'enses', case=False)) | (latlib_df.text.str.contains(pat = 'ensium', case=False)) | (latlib_df.text.str.contains(pat = 'ensibus', case=False))]

In [13]:
#capulus:
capulus = latlib_df[(latlib_df.text.str.contains(pat = 'capulus', case=False)) | (latlib_df.text.str.contains(pat = 'capuli', case=False)) | (latlib_df.text.str.contains(pat = 'capulo', case=False)) | (latlib_df.text.str.contains(pat = 'capulum', case=False)) | (latlib_df.text.str.contains(pat = 'capulorum', case=False)) | (latlib_df.text.str.contains(pat = 'capulis', case=False)) | (latlib_df.text.str.contains(pat = 'capulos', case=False))]

In [14]:
#let's see how many occurences of each word there are:
print('gladius: ',len(gladius))
print('ferrum: ',len(ferrum))
print('mucro: ',len(mucro))
print('ensis: ',len(ensis))
print('capulus: ',len(capulus)) #'capulus' is the least common, only appearing 48 times, let's look at tht data!

gladius:  277
ferrum:  462
mucro:  89
ensis:  573
capulus:  48


In [15]:
capulus

,title,text,text_raw,tokens,token_count
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524
25,Statius: Achilleid II,PVBLIVS PAPINIVS STATIVS ACHILLEID II Exuit im...,"[PVBLIVS PAPINIVS STATIVS, \nACHILLEID II\n, \...","[PVBLIVS, PAPINIVS, STATIVS, ACHILLEID, II, Ex...",1243
31,Statius: Thebaid XI,PVBLIVS PAPINIVS STATIVS THEBAID XI Postquam m...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID XI\n, \nP...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, XI, Post...",6017
34,Statius: Thebaid XII,PVBLIVS PAPINIVS STATIVS THEBAID XII Nondum cu...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID XII\n, \n...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, XII, Non...",6244
35,Statius: Thebaid IX,PVBLIVS PAPINIVS STATIVS THEBAID IX Asperat Ao...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID IX\n, \nA...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, IX, Aspe...",6925
36,Statius: Thebaid VIII,PVBLIVS PAPINIVS STATIVS THEBAID VIII Vt subit...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID VIII\n, \...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, VIII, Vt...",5870
37,Statius: Thebaid X,PVBLIVS PAPINIVS STATIVS THEBAID X Obruit Hesp...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID X\n, \nOb...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, X, Obrui...",7200
38,Statius: Thebaid III,PVBLIVS PAPINIVS STATIVS THEBAID III At non Ao...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID III\n, \n...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, III, At,...",5515
39,Statius: Thebaid V,PVBLIVS PAPINIVS STATIVS THEBAID V Pulsa sitis...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID V\n, \nPu...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, V, Pulsa...",5822
40,Statius: Thebaid IV,PVBLIVS PAPINIVS STATIVS THEBAID IV Tertius ho...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID IV\n, \nT...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, IV, Tert...",6424


Just from looking at these titles there's a pretty fair distribution of genre between the works (some philosophy, epic poetry, history, and even comedy plays!)

In [16]:
gladius.head()

,title,text,text_raw,tokens,token_count
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918


let's work on seperating works by style (prose or poetry):

In [17]:
#use Aeneid (P. VERGILI MARONIS AENEIDOS) + Catullus (C. VALERIVS CATVLLVS) for verse examples

In [25]:
metamorphoses_ovid = latlib_df[latlib_df['text'].str.contains('P. OVIDI NASONIS METAMORPHOSEON')]

In [24]:
aeneid = latlib_df[latlib_df['text'].str.contains('P. VERGILI MARONIS AENEIDOS')]